<a href="https://colab.research.google.com/github/andresenrique08/ML_2020/blob/master/Reto%202/Bono2_ML_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bono - Reto 2

Autor:
*   Andrés Enrique Ávila Rojas (201319127)

**Gradiente descendiente estocástico**


In [1]:
# Import libraries 
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

Se toman los datos del repositorio de github con el siguiente link https://github.com/andresenrique08/ML_2020/blob/master/Reto%202/weatherAUS.csv

In [2]:
# Reading file from github
url = "https://raw.githubusercontent.com/andresenrique08/ML_2020/master/Reto%202/weatherAUS.csv"
data  = pd.read_csv(url)
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [3]:
data.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RISK_MM          float64
RainTomorrow      object
dtype: object

## Selección de variables númericas

In [4]:
# Se seleccionan las variables de tipo númerico
data_num = [var for var in data.columns if data[var].dtype!='O']
data[data_num].head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RISK_MM
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0.0
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0.0
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0.0
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,1.0
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.2


## Selección de variables númericas



Ahora se crea un nuevo data frame con las variables númericas y la variable objetivo, para posteriormente hacer la limpieza de los datos

In [5]:
data_main = data[data_num+['RainTomorrow']]
data_main.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RISK_MM,RainTomorrow
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0.0,No
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0.0,No
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0.0,No
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,1.0,No
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.2,No


Se verifican los valores nulos dentro del dataframe resultante

In [6]:
def missing_values_table(df): 
        val_per = df.isnull().sum()
        porcentaje_val_per = 100 * df.isnull().sum()/len(df)
        tabla_val_per = pd.concat([val_per, porcentaje_val_per], axis=1)
        tabla_val_per_ren_columnas = tabla_val_per.rename(
        columns = {0 : 'Valores perdidos', 1 : '% sobre el total'})
        return tabla_val_per_ren_columnas 

In [7]:
missing_values_table(data_main)

,Valores perdidos,% sobre el total
MinTemp,637,0.447983
MaxTemp,322,0.226453
Rainfall,1406,0.988797
Evaporation,60843,42.789026
Sunshine,67816,47.692924
WindGustSpeed,9270,6.519308
WindSpeed9am,1348,0.948007
WindSpeed3pm,2630,1.849599
Humidity9am,1774,1.247600
Humidity3pm,3610,2.538803


Se elimina la variable `RISK_MM`, puesto que podría filtrar información al modelo y reducir el porcentaje de acierto en la predicción. Más información en https://www.kaggle.com/jsphyg/weather-dataset-rattle-package.

In [8]:
data_main = data_main.drop(columns=['RISK_MM'],axis=1)

Se procede a elimininar todas las variables con un porcentaje de datos perdidos superior al 30%, consideandose un porcentaje alto de pérdidas.

In [9]:
# Se define la función para eliminar las variables del dataframe df con porcentaje p
def eliminar_var(df,p):
    daticos = missing_values_table(df)
    names = list(df.columns)
    for i in range(len(daticos)):
        if daticos.iloc[i,1]>p:
            df.drop(names[i], axis=1, inplace=True)

In [10]:
# Se eliminan las variables con 30% de datos perdidos
eliminar_var(data_main,30)
# Se muestran las variables que quedan para el análisis y su porcentaje de datos perdidos
data_main.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainTomorrow
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No


Se eliminan las filas de las variables perdidas, realizando la limpieza completa de los dato

In [11]:
data_main=data_main.dropna(how="any")
missing_values_table(data_main)

,Valores perdidos,% sobre el total
MinTemp,0,0.0
MaxTemp,0,0.0
Rainfall,0,0.0
WindGustSpeed,0,0.0
WindSpeed9am,0,0.0
WindSpeed3pm,0,0.0
Humidity9am,0,0.0
Humidity3pm,0,0.0
Pressure9am,0,0.0
Pressure3pm,0,0.0


Se observa que el dataframe ya no tiene valores perdidos, ahora se procede a verificar su tamaño

In [12]:
data_main.shape

(119590, 13)

## Selección de datos (Entrenamiento, prueba)

`X` contiene los datos disponibles para la predicción y `Y` contiene el vector de valores de la variable objetivo, `RainTomorrow` en este caso.

In [13]:
X = data_main.drop(['RainTomorrow'], axis=1)
Y = data_main['RainTomorrow']

Se estandarizan los valores de X

In [14]:
# Se usa el minmaxscaler
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns) 
X.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm
0,0.513064,0.446154,0.001632,0.294574,0.229885,0.275862,0.71,0.22,0.449587,0.4800,0.461894,0.446667
1,0.370546,0.494505,0.000000,0.294574,0.045977,0.252874,0.44,0.25,0.497521,0.4912,0.468822,0.502222
2,0.501188,0.507692,0.000000,0.310078,0.218391,0.298851,0.38,0.30,0.447934,0.5056,0.556582,0.477778
3,0.413302,0.558242,0.000000,0.139535,0.126437,0.103448,0.45,0.16,0.613223,0.5712,0.489607,0.551111
4,0.610451,0.652747,0.002720,0.271318,0.080460,0.229885,0.82,0.33,0.500826,0.4624,0.482679,0.622222


Se separan los datos en entrenamiento y prueba

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
# Comparación de tamaños
X_train.shape, X_test.shape

((95672, 12), (23918, 12))

## Implementación de Gradiente descendiente estocástico

Para esta implementación se usará la clase `SGDClassifier` de la librería Sklearn, que implementa la rutina de graficente descendiente estocástico y soporta varias funciones de pérdida. En este caso se usará como función de pérdida la entropía cruzada. 

Teniendo en cuenta la naturaleza estandarizada de `X`, tendremos que `Y` es una variable aleatoria de Bernoulli. En casos donde se tiene una distribución de probabilidad de Bernoulli, calcular la entropía cruzada dará el mismo resultado que calcular una perdida logaritimica (Libro: Página 246, Machine Learning: A Probabilistic Perspective, 2012). Por tanto, se usará la función `"log"`, de la clase `SGDClassifier` presentado a continuación:

In [16]:
SGD_class = SGDClassifier(loss="log")
SGD_class.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

## Resultados de predicción

Se crea el vector con los valores predecidos

In [17]:
# Se ubican los valores obtenidos en un vector
y_pred_test = SGD_class.predict(X_test)

Ahora, se evaluan los resultados obtenidos. Para esto se usará el puntaje de precisión de clasificación (accuracy_score) de sklearn.




In [18]:
# Encontrando el puntaje de precisión
print('Puntaje de exactitud del modelo: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Puntaje de exactitud del modelo: 0.8480


Se observa un porcentaje de presición del 84.79%, por lo que se puede concluir que el modelo puede ser mejorado, para ello se necesita un análisis estadístico como el de la primera actividad de este reto.